In [1]:
# import gym
# env = gym.make('Reacher-v2')
# env.reset()
# env.render()
# env.close()

Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Creating window glfw


In [1]:
from ilqr import iLQR
import gym
import numpy as np

from aprl.agents import MujocoFiniteDiffDynamics, MujocoFiniteDiffCost

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Logging to /tmp/openai-2019-01-25-15-24-42-534265
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']


In [2]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    info = "converged" if converged else ("accepted" if accepted else "failed")
    print("iteration", iteration_count, info, J_opt, xs[-1])

In [3]:
# Environment setup
env = gym.make('Reacher-v2').unwrapped
env.seed(42)
env.reset()
dynamics = MujocoFiniteDiffDynamics(env)
x0 = dynamics.get_state()
N = 100  # planning horizon
us_init = np.array([env.action_space.sample() for _ in range(N)])

# Finite difference cost

In [ ]:
finite_cost = MujocoFiniteDiffCost(env)
finite_ilqr = iLQR(dynamics, finite_cost, N)

In [ ]:
finite_xs, finite_us = finite_ilqr.fit(x0, us_init, n_iterations=100, on_iteration=on_iteration)

# Analytic cost

In [4]:
import theano
from theano import tensor as T
from ilqr.cost import AutoDiffCost

# Reacher, Gym observation:
# obs[0:1]: xs; np.cos(qpos[0:2]) (qpos[0] is joint0, qpos[1] is joint1)
# obs[2:3]: ys; np.sin(qpos[0:2]);
# obs[4:5]: goal x and y; qpos[2:]; (target_x and target_y)
# obs[6:7]: theta dot
# obs[8:9]: xy of fingertip - target

# TODO: why not gym obs? (Think I tried this and decided against it)
# TODO: do we need/want timestep?

def make_reacher_cost(kind, control_weight=1.0):
    # qpos[0:3]: theta of joint 0, theta of joint 1; target x and y.
    qpos_inputs = [T.dscalar('theta'), T.dscalar('phi'), T.dscalar('targetx'), T.dscalar('targety')]
    # qvel: derivatives of the above; note target x and y are constant so have derivative zero.
    qvel_inputs = [T.dscalar('thetadot'), T.dscalar('phidot'), T.dscalar('_zero1'), T.dscalar('_zero2')]
    # qacc: second derivatives of qpos. We don't actually use these in the cost.
    qacc_inputs = [T.dscalar('_acc{}'.format(i)) for i in range(len(qpos_inputs))]
    # qacc_warmstart: same shape as qacc
    qacc_warmstart_inputs = [T.dscalar('_accwarm{}'.format(i)) for i in range(len(qpos_inputs))]
    if kind == 'mjsimstate':
        # Reacher, MJSimState.flatten():
        # obs[0]: time step, obs[1:4]: qpos[0:3]; obs[5:8]: qvel[0:3]
        # In general might include action and udd_state, but not for Reacher.
        x_inputs = [T.dscalar('_time')] + qpos_inputs + qvel_inputs
    elif kind == 'mujocorelevantstate':
        # Reacher, MujocoRelevantState.flatten()
        x_inputs = qpos_inputs + qvel_inputs + qacc_inputs + qacc_warmstart_inputs
    else:
        raise ValueError("Unrecognised kind: '{}'".format(kind))
    u_inputs = [T.dscalar('thetadotdot'), T.dscalar('phidotdot')]
    qpos = T.stack(qpos_inputs)
    u = T.stack(u_inputs)
    
    control_cost = T.dot(u, u)
    target_xpos = qpos[2:4]
    body1_xpos = 0.1 * T.stack([T.cos(qpos[0]), T.sin(qpos[1])])
    fingertip_xpos_delta = 0.11 * T.stack([T.cos(qpos[1]), T.sin(qpos[1])])
    fingertip_xpos = body1_xpos + fingertip_xpos_delta
    delta = fingertip_xpos - target_xpos
    state_cost = T.sqrt(T.dot(delta, delta))
    l = state_cost + control_weight * control_cost
    l_terminal = T.zeros(())
    return AutoDiffCost(l, l_terminal, x_inputs, u_inputs)

In [11]:
analytic_cost = make_reacher_cost('mjsimstate')
analytic_ilqr = iLQR(dynamics, analytic_cost, N)

In [12]:
analytic_xs, analytic_us = analytic_ilqr.fit(x0, us_init, n_iterations=100, on_iteration=on_iteration)

iteration -1 converged 79.96275887661503 [ 2.          7.77702008  1.21356644  0.02243766  0.07369059  3.36928471
 -8.26249354  0.          0.        ]
iteration 0 accepted 29.265523052069145 [ 2.          7.70043922  1.385693    0.02243766  0.07369059  3.60590171
 -8.192679    0.          0.        ]
iteration 1 accepted 23.25980857049346 [ 2.          7.54924921  1.55563349  0.02243766  0.07369059  3.85633321
 -8.08002884  0.          0.        ]
iteration 2 accepted 20.571476283879594 [ 2.          7.48510752  1.91564321  0.02243766  0.07369059  4.1656226
 -7.47378228  0.          0.        ]
iteration 3 accepted 19.314515201704687 [ 2.          7.05905434  2.27970603  0.02243766  0.07369059  3.3447239
 -5.31276474  0.          0.        ]
iteration 4 accepted 19.270514329106753 [ 2.          6.82783495  2.63384984  0.02243766  0.07369059  1.71902347
 -2.00996315  0.          0.        ]
iteration 5 accepted 17.247589853076487 [ 2.          6.52346203  2.69115363  0.02243766  0.0736

# Alternative FD

In [5]:
from aprl.agents.mujoco_control import MujocoFiniteDiffDynamicsLowLevel
better_dynamics = MujocoFiniteDiffDynamicsLowLevel(env)
better_x0 = better_dynamics.get_state()
better_analytic_cost = make_reacher_cost('mujocorelevantstate')
better_analytic_ilqr = iLQR(better_dynamics, better_analytic_cost, N)
better_analytic_xs, better_analytic_us = better_analytic_ilqr.fit(better_x0, us_init, n_iterations=100, on_iteration=on_iteration)

iteration -1 converged 79.96275887661503 [ 7.77702008e+00  1.21356644e+00  2.24376640e-02  7.36905865e-02
  3.36928471e+00 -8.26249354e+00  0.00000000e+00  0.00000000e+00
 -1.80000140e+02 -1.79251765e+01  0.00000000e+00  0.00000000e+00
 -1.80000140e+02 -1.79251765e+01  0.00000000e+00  0.00000000e+00]
iteration 0 accepted 79.3541596217529 [ 7.77051152e+00  1.21481979e+00  2.24376640e-02  7.36905865e-02
  3.36582069e+00 -8.25763157e+00  0.00000000e+00  0.00000000e+00
 -1.79995963e+02 -1.79262821e+01  0.00000000e+00  0.00000000e+00
 -1.79995963e+02 -1.79262821e+01  0.00000000e+00  0.00000000e+00]
iteration 1 accepted 79.26610987757593 [ 7.74641531e+00  1.21524421e+00  2.24376640e-02  7.36905865e-02
  3.34649536e+00 -8.25581721e+00  0.00000000e+00  0.00000000e+00
 -1.79986737e+02 -1.79249592e+01  0.00000000e+00  0.00000000e+00
 -1.79986737e+02 -1.79249592e+01  0.00000000e+00  0.00000000e+00]
iteration 2 accepted 79.2601534342358 [ 7.72579799e+00  1.21428814e+00  2.24376640e-02  7.36905865e

iteration 33 accepted 24.546389897578482 [ 17.63971951   2.20571726   0.02243766   0.07369059  13.41329664
  -1.11854826   0.           0.         -13.84616121   1.30026141
   0.           0.         -13.84616121   1.30026141   0.
   0.        ]
iteration 34 accepted 24.36416474473091 [ 16.19893132   1.87984618   0.02243766   0.07369059  11.32768013
  -1.69257883   0.           0.         -13.79354211   1.31322168
   0.           0.         -13.79354211   1.31322168   0.
   0.        ]
iteration 35 failed 24.364164744730903 [ 16.19893132   1.87984618   0.02243766   0.07369059  11.32768013
  -1.69257883   0.           0.         -13.79354211   1.31322168
   0.           0.         -13.79354211   1.31322168   0.
   0.        ]
iteration 36 failed 24.364164744730903 [ 16.19893132   1.87984618   0.02243766   0.07369059  11.32768013
  -1.69257883   0.           0.         -13.79354211   1.31322168
   0.           0.         -13.79354211   1.31322168   0.
   0.        ]
iteration 37 accepted

# Receding horizon

In [ ]:
from ilqr.controller import RecedingHorizonController

controller = RecedingHorizonController(x0, analytic_ilqr)  # can also use finite_ilqr
rew = []
receding_xs = []
receding_us = []
for x, u in controller.control(us_init, subsequent_n_iterations=10):
    ob, r, done, info = env.step(u)
    receding_xs.append(x)
    receding_us.append(u)
    rew.append(r)
    print('iteration', len(rew), r, x, u)
    if len(rew) == 50:
        break

# Rollouts

In [7]:
import time

def rollout(env, dynamics, x0, us, render=False):
    dynamics.set_state(x0)
    if render:
        env.render()
    rew = []
    actual_xs = []
    for u in us:
        _obs, r, done, info = env.step(u)
        rew.append(r)
        actual_xs.append(dynamics.get_state())
        assert not done
        if render:
            env.render()
            time.sleep(0.05)
    return rew, actual_xs

In [18]:
rew, actual_xs = rollout(env.unwrapped, dynamics, x0, analytic_us, render=True)
#rew, actual_xs = rollout(env.unwrapped, better_dynamics, better_x0, better_analytic_us, render=True)
print(sum(rew))